In [ ]:
import time, re, os, shutil
import json
import numpy as np
import pandas as pd
# from numpyencoder import NumpyEncoder
from string import punctuation
import urllib.request
import requests

from pathlib import Path
import difflib

import pandas as pd
pd.options.mode.chained_assignment = None

In [ ]:
api_1 = 'https://xxxxxxxxxx.in/api/vendor/TestDetailsKeywords'

def GetTestDetailsKeywords(url):
    #print("In GetTestDetailsKeywords func")
    # print(url)
    data = None
    try:
        resp = requests.get(url).json()
        data = resp['Data']
        #print(resp['Status'])
        #print(resp['Message'])
    except Exception as exception:
        print(f"Name of Exception: {exception}")
    return data

def get_keywords_list():
    print("--- Fetch Alt keywords.....")
    GetTestDetailsKeywords_data = GetTestDetailsKeywords(api_1)
    if GetTestDetailsKeywords_data:
        #print(GetTestDetailsKeywords_data)
        pass
    else:
        GetTestDetailsKeywords_data = []
        print("No GetTestDetailsKeywords_data found")
    return GetTestDetailsKeywords_data

def MLGetAHCReportURL(url):
    #print("In MLGetAHCReportURL func")
    #print(url)
    data = None
    try:
        resp = requests.get(url).json()
        data = resp['Data']
        #print(resp['Status'])
        #print(resp['Message'])
    except Exception as exception:
        print(f"Name of Exception: {exception}")
    return data


# function converting keywords data to our compatible format
def formatKeywordsDataAndsaveFile(kwrds_data):
    formatted_data = [ {"GroupName": dict_itm['GroupName'],
                        "TestID": dict_itm['TestID'],
                        "TestName": dict_itm['TestName'],
                        "UnitID": dict_itm['UnitID'],
                        "UnitName": dict_itm['UnitName'],
                        "RangeMale": dict_itm['RangeMale'],
                        "RangeFemale": dict_itm['RangeFemale'],
                        "AltTestID": dict_itm['AltTestID'],
                        "AltUnitID": dict_itm['AltUnitID'],
                        "AltTestNamesList": [ alt_name_dict['TestName'].strip() for alt_name_dict in dict_itm['AlternateTestNameList'] ] if dict_itm['AlternateTestNameList'] else [].copy(),
                        "AltTestIDList": [ alt_name_dict['AltTestID'] for alt_name_dict in dict_itm['AlternateTestNameList'] ] if dict_itm['AlternateTestNameList'] else [].copy(),
                        "AltUnitNamesList": [ alt_name_dict['UnitName'].strip() for alt_name_dict in dict_itm['AlternateUnitNameList'] ] if dict_itm['AlternateUnitNameList'] else [].copy(),
                        "AltUnitIDList": [ alt_name_dict['AltUnitID'] for alt_name_dict in dict_itm['AlternateUnitNameList'] ] if dict_itm['AlternateUnitNameList'] else [].copy(),
                        } for dict_itm in kwrds_data ]
    df = pd.DataFrame(formatted_data)
    df['AltTestNamesStr'] = df['AltTestNamesList'].apply(lambda x: '; '.join(x))
    df['AltUnitNamesStr'] = df['AltUnitNamesList'].apply(lambda x: '; '.join(x))
    df['AltTestIDcount'] = df['AltTestIDList'].apply(len)
    df['AltUnitIDcount'] = df['AltUnitIDList'].apply(len)
    df = df[['GroupName', 'TestID', 'TestName', 'UnitID', 'UnitName',  'AltTestNamesStr','AltTestIDcount',
           'AltUnitNamesStr',  'AltUnitIDcount', 'RangeMale','RangeFemale', 'AltTestID', 'AltUnitID','AltTestNamesList',
           'AltTestIDList', 'AltUnitNamesList', 'AltUnitIDList', ]]
    df.sort_values('TestID', ignore_index = True,inplace= True,)
    timestamp11 = time.strftime('%d%m%y', time.localtime())
    # path_ = f"AllKeywords_{timestamp11}.xlsx"
    path_ = f"AllKeywords_{timestamp11}.pkl"
    # df.to_excel(path_, index=False)
    df.to_pickle(path_, )
    print("_______"*10)
    print("Total number of group: ", len(set(df['GroupName'].values)))
    print("Total number of TestNames: ", len(set(df['TestName'].values)))
    print("_______"*10)
    group_counts = df.groupby('GroupName').size().reset_index(name='Count').sort_values(by='Count', ascending=False).reset_index(drop=True)
    print("GroupName with count: ")
    print(group_counts)
    print("_______"*10)
    return True

In [ ]:
keywords_api_data = get_keywords_list()
# assert keywords_api_data, "Failed to get keywords data! Program aborted."
fg = formatKeywordsDataAndsaveFile(keywords_api_data)
fg

--- Fetch Alt keywords.....
______________________________________________________________________
Total number of group:  17
Total number of TestNames:  68
______________________________________________________________________
GroupName with count: 
                                  GroupName  Count
0                CBC (COMPLETE BLOOD COUNT)     15
1     LIVER  PROFILE (CARDIAC RISK PROFILE)     11
2      LIPID PROFILE (CARDIAC RISK PROFILE)      8
3      KIDNEY PROFILE (RENAL FUNCTION TEST)      6
4   THYROID PROFILE (THYROID FUNCTION TEST)      5
5                            DIABETIC PANEL      4
6                              IRON PROFILE      4
7                            URINE ANALYSIS      3
8                              ELECTROLYTES      3
9                           VITAMIN PROFILE      2
10                   PROSTATE CANCER MARKER      1
11                           BLOOD PRESSURE      1
12                        BODY MEASUREMENTS      1
13                     FEMALE CANCE

True

In [ ]:
pt = '/content/AllKeywords_100424.pkl'
df = pd.read_pickle(pt)
df = df[['GroupName', 'TestID', 'TestName', 'UnitName', 'AltTestNamesList', 'AltUnitNamesList']]
print(df.shape)

(68, 6)

In [ ]:
pt = '/content/AllKeywords_100424.pkl'
df = pd.read_pickle(pt)
df = df[['GroupName', 'TestID', 'TestName', 'UnitName', 'AltTestNamesList', 'AltUnitNamesList']]
print(df.shape)

def get_data(TestID):
    idx_ = df[df['TestID']==TestID].index[0]
    data = {}
    data['TestID'] = TestID
    d2 = df[['TestName', 'AltTestNamesList', 'UnitName', 'AltUnitNamesList']]
    for i in range(len(d2)):
        TestName = d2.iloc[i, 0]
        AltTestNamesList = d2.iloc[i, 1]
        if TestName not in AltTestNamesList:
            TestName_list = [d2.iloc[i, 0]]
            TestName_list.extend(AltTestNamesList)
        else:
            TestName_list = AltTestNamesList

        UnitName = d2.iloc[i, 2]
        AltUnitNamesList = d2.iloc[i, 3]
        if UnitName not in AltUnitNamesList:
            UnitName_list = [d2.iloc[i, 2]]
            UnitName_list.extend(AltUnitNamesList)
        else:
            UnitName_list = AltUnitNamesList

        d2.iloc[i, 1] =  TestName_list
        d2.iloc[i, 3] =  UnitName_list

    AltTestNamesList  = list(d2.iloc[idx_, 1])
    # print(len(AltTestNamesList), AltTestNamesList )
    AltUnitNamesList  = list(d2.iloc[idx_, 3])
    # print(len(AltUnitNamesList), AltUnitNamesList)
    # print("Total outcomes: ", (len(AltTestNamesList)*len(AltUnitNamesList)))
    data['AltTestNamesList'] = AltTestNamesList
    data['AltUnitNamesList'] = AltUnitNamesList
    data['TotalOutcomes'] = (len(AltTestNamesList)*len(AltUnitNamesList))
    return data

In [ ]:
test_id = 25
data = get_data(test_id )
data

In [ ]:
sent = f"$TNTNTN$	120/82 MMHG	$UTUTUT$"

$TNTNTN$
$UTUTUT$

In [ ]:


# 1: BMI sentance not found

# 2: BP	120/82 MMHG	mm/Hg
# sent = f"{test_nm}	120/82 MMHG	{unit}"

# 3:    RBC COUNT	5.21	Million/cu.mm	4.5-5.5	Electrical Impedence
# sent = f"{test_nm}	5.21	{unit}	4.5-5.5	Electrical Impedence"

# 4: HEMOGLOBIN (HB)	13.8	13.0 - 17.0	g/dL
# sent = f"{test_nm}	13.8	13.0 - 17.0	{unit}"

# 5: HEMATOCRIT (PCV)	43.6	40 - 50	%
# sent = f"{test_nm}	43.6	40 - 50	{unit}"

# 6: MEAN CORPUSCULAR VOLUME (MCV)	92.0	83 - 101	fL
# sent = f"{test_nm}	92.0	83 - 101	{unit}"

# 7: MCHC (Mean Corpuscular Hemoglobin	30.5	g/dl	31.5-34.5
# sent = f"{test_nm}	30.5	{unit}	31.5-34.5"

# 8: RED CELL DISTRIBUTION WIDTH (RDW)	13.7	11.6 - 14.0	%
# sent = f"{test_nm}	13.7	11.6 - 14.0	{unit}"

# 9: RDW-SD	40.70	FL	39.0-46.0
# sent = f"{test_nm}	40.70	{unit}	39.0-46.0"

# 10: WHITE BLOOD CELL COUNT	6.38	4.0 - 10.0	thou/µL
# sent = f"{test_nm}	6.38	4.0 - 10.0	{unit}"

# 11: NEUTROPHILS	59	40 - 80	%
# sent = f"{test_nm}	59	40 - 80	{unit}"

# 12: LYMPHOCYTES	33	20 - 40	%
# sent = f"{test_nm}	33	20 - 40	{unit}"

# 13: MONOCYTES	7	2.0 - 10.0	%
# sent = f"{test_nm}	7	2.0 - 10.0	{unit}"

# 14: EOSINOPHILS	1	1.0 - 6.0	%
# sent = f"{test_nm}	1	1.0 - 6.0	{unit}"

# 15: BASOPHILS	0	0 - 2	%
# sent = f"{test_nm}	0	0 - 2	{unit}"

# 16: PLATELET COUNT	223000	cells/cu.mm	150000-410000	Electrical impedence
# sent = f"{test_nm}	223000	{unit}	150000-410000	Electrical impedence"

# E.S.R (WINTROBE)	26	mm/1st	0-9 ?
# 17: ERYTHROCYTE SEDIMENTATION	3	mm at the end	0-15	Modified Westergren
# sent = f"{test_nm}	3	{unit}	0-15	Modified Westergren"

# 18: BLOOD SUGAR FASTING	98.0	mg/dl	70-110
# sent = f"{test_nm}	98.0	{unit}	70-110"

# 19: GLUCOSE, POST PRANDIAL (PP), 2	103	mg/dL	70-140	HEXOKINASE
# sent = f"{test_nm} 2	103	{unit}	70-140	HEXOKINASE"

# 20: 'Random Blood Sugar'
# "Random Blood Sugar (RBS)-Plasma 2	103	{unit}	70-140"
# sent = f"{test_nm} 2	103	{unit}	70-140"

# 21: 'GLYCOSYLATED HEMOGLOBIN (HBA1C)	5.6	Non-diabetic: < 5.7	%'
# sent = f"{test_nm}	5.6	Non-diabetic: < 5.7	{unit}"

# 22: CHOLESTEROL	141	< 200 Desirable	mg/dL
# sent = f"{test_nm}	141	< 200 Desirable	{unit}"

# 23: HDL CHOLESTEROL	36	Low	Low HDL Cholesterol <40	mg/dL
# sent = f"{test_nm}	36	Low	Low HDL Cholesterol <40	{unit}"

# 24: DIRECT LDL CHOLESTEROL	93.00	Optimal : < 100	mg/dL
# sent = f"{test_nm}	93.00	Optimal : < 100	{unit}"

# 25: TRIGLYCERIDES	245	mg/dL	<150	Enzymatic
# sent = f"{test_nm}	245	{unit}	<150	Enzymatic"

# 26: 'LDL / HDL Ratio',
# sentance not found
# 27: 'TC / HDL Ratio',
# sentance not found

# 28:  'Non HDL Cholesterol',
# f"Non HDL Cholesterol	123	Optimal : <130 mg/dL"
# f"{test_nm}	123	Optimal : <130 {unit}"

# 29:  'Total Triiodothyronine',
# Triiodothyronine Total (T3)	1.16	0.80-2.00 ng/mL
# f"{test_nm}	1.16	0.80-2.00 {unit}"

# 30:   'Total Thyroxine'
# Thyroxine Total (T4)	8.20	4.6-12.0 µg/dL
# f"{test_nm}	8.20	4.6-12.0 {unit}"

# 31:Thyroid Stimulating Hormone
# Thyroid Stimulating Hormone (TSH)	1.74	0.27-4.20 µIU/mL
# {test_nm}	1.74	0.27-4.20 {unit}


# 32: 'BUN',
# Blood Urea Nitrogen-BUN*	17.34	7 - 18	mg/dL
# {test_nm}	17.34	7 - 18	{unit}

# 33:'Creatinine',
# Creatinine*	0.67	0.67 - 1.17	mg/dL
# {test_nm}	0.67	0.67 - 1.17	{unit}

# 34: 'Uric Acid',
# Uric Acid*	5.60	3.5 - 7.2	mg/dL
# {test_nm}	5.60	3.5 - 7.2	{unit}

# 35: 'Urea',
# Urea *	37.10	17- 43	mg/dL
# {test_nm}	37.10	17- 43	{unit}

# 36: 'Calcium',
# Calcium*	9.70	8.8 - 10.6	mg/dL
# {test_nm}	9.70	8.8 - 10.6	{unit}

# 37: 'Phosphorus',
# Phosphorus,serum	2.39	mg/dL	2.6-4.5
# {test_nm}	2.39	{unit}	2.6-4.5

# 38 '25 - Hydroxy Vitamin D',
# 25 Hydroxy Vitamin D	33.68	Deficiency: < 20 ng/mL
# {test_nm}	33.68	Deficiency: < 20 {unit}

# 39: 'Vitamin B12',
# VITAMIN B12 , SERUM	143	pg/mL	107.2-653.3	CLIA
# {test_nm}	143	{unit}	107.2-653.3	CLIA

# 40: 'Total Iron Binding Capacity',
# Total Iron Binding Capacity (TIBC)	291	158-538 µg/dL
# {test_nm}	291	158-538 {unit}

# 41: 'Sr. Iron'
# Iron	88.6	33-193 µg/dL
# {test_nm}	88.6	33-193 {unit}

# 42      % Transferrin Saturation
# Transferrin Saturation	6.88	%	15 - 45
# {test_nm}	6.88	{unit}	15 - 45

# 43                        Sodium
# Sodium	140	136-149 mmol/L
# {test_nm}	140	136-149 {unit}

# 44                     Potassium
# Potassium	4.4	3.5-5.3 mmol/L
# {test_nm}	4.4	3.5-5.3 {unit}

# 45                      Chloride
# Chloride, serum	102.2	meq/l	98-107
# {test_nm}	102.2	{unit}	98-107

# 46   Aspartate Aminotransferase
# Aspartate aminotransferase(AST/SGOT)	19.0	IU/L	<35
# {test_nm}	19.0	{unit}	<35

# 47          Alanine Transaminase
# Alanine aminotransferase(ALT/SGPT)	27.0	U/L	<45
# {test_nm}	27.0	{unit}	<45


# 48    Gamma Glutamyl Transferase
# GGTP(Gamma GT,Serum 24.0	U/L	7-50
# {test_nm} 24.0	{unit}	7-50

# 49              Bilirubin Direct
# Bilirubin (Direct)	0.10	mg/dl	0.1-0.5
# {test_nm}	0.10	{unit}	0.1-0.5

# 50            Bilirubin Indirect
# Bilirubin (Indirect)	0.30	mg/dL	0.1-1.0
# {test_nm}	0.30	{unit}	0.1-1.0

# 51               Bilirubin Total
# Bilirubin (Total)	0.40	mg/dl	0.2-1.1
# {test_nm}	0.40	{unit}	0.2-1.1

# 52          Alkaline Phosphatase
# Alkaline Phosphatase	76	U/L	32-91
# {test_nm}	76	{unit}	32-91


# 53                     Albumin Serum
# Albumin	4.5	g/dl	3.5-5.0
# {test_nm}	4.5	{unit}	3.5-5.0


# 54                    Globulin Serum
# Globulin	2.5	g/dl	2.3-3.5
# {test_nm}	2.5	{unit}	2.3-3.5


# 55    Serum Albumin : Globulin Ratio
# "sentance not found",

# 56                     Protein Total
# Total Protein	7.00	g/dl	6.5-8.1
# {test_nm}	7.00	{unit}	6.5-8.1

# 61                            CA 125
# Total Protein	7.00	g/dl	6.5-8.1
# {test_nm}	7.00	{unit}	6.5-8.1

# 63                              PSA
# PSA-Total (Prostate-specific antigen-Total)	0.89	ng/ml	<2.5
# {test_nm}	0.89	{unit}	<2.5

# 64      Mean Corpuscular Haemoglobin
# MCH (Mean Corpuscular Hemoglobin )	31.7	pg	27-32
# {test_nm}	31.7	{unit}	27-32

# 65                  VLDL Cholesterol
# VLDL CHOLESTEROL	42.6	mg/d L	10 - 40
# {test_nm}	42.6	{unit}	10 - 40

# 67                              CRP
# VLDL CHOLESTEROL	42.6	mg/d L	10 - 40
# {test_nm}	42.6	{unit}	10 - 40

# 68                            HSCRP
# VLDL CHOLESTEROL	42.6	mg/d L	10 - 40
# {test_nm}	42.6	{unit}	10 - 40

# 69                          Ferritin
# Ferritin	184.18	22-322 ng/mL
# {test_nm}	184.18	22-322 {unit}

# 77           Pus cells
# PUS CELLS 1-2 /hpf 0-5 Microscopy
# {test_nm} 1-2 {unit} 0-5 Microscopy

# 78    Epithelial cells
# EPITHELIAL CELLS 2-4 /hpf <10 MICROSCOPY
# {test_nm} 2-4 {unit} <10 MICROSCOPY

# 79                 RBC
# RBC NIL /hpf 0-2 MICROSCOPY
# {test_nm} NIL {unit} 0-2 MICROSCOPY

# 80                 FT3


# 81                 FT4

In [ ]:
pt = '/content/AllKeywords_100424.pkl'
df = pd.read_pickle(pt)
df = df[['GroupName', 'TestID', 'TestName', 'UnitName', 'AltTestNamesList', 'AltUnitNamesList']]
print(df.shape)

def get_data(TestID):
    idx_ = df[df['TestID']==TestID].index[0]
    data = {}
    data['TestID'] = TestID
    d2 = df[['TestName', 'AltTestNamesList', 'UnitName', 'AltUnitNamesList']]
    for i in range(len(d2)):
        TestName = d2.iloc[i, 0]
        AltTestNamesList = d2.iloc[i, 1]
        if TestName not in AltTestNamesList:
            TestName_list = [d2.iloc[i, 0]]
            TestName_list.extend(AltTestNamesList)
        else:
            TestName_list = AltTestNamesList

        UnitName = d2.iloc[i, 2]
        AltUnitNamesList = d2.iloc[i, 3]
        if UnitName not in AltUnitNamesList:
            UnitName_list = [d2.iloc[i, 2]]
            UnitName_list.extend(AltUnitNamesList)
        else:
            UnitName_list = AltUnitNamesList

        d2.iloc[i, 1] =  TestName_list
        d2.iloc[i, 3] =  UnitName_list

    AltTestNamesList  = list(d2.iloc[idx_, 1])
    # print(len(AltTestNamesList), AltTestNamesList )
    AltUnitNamesList  = list(d2.iloc[idx_, 3])
    # print(len(AltUnitNamesList), AltUnitNamesList)
    # print("Total outcomes: ", (len(AltTestNamesList)*len(AltUnitNamesList)))
    data['AltTestNamesList'] = AltTestNamesList
    data['AltUnitNamesList'] = AltUnitNamesList
    data['TotalOutcomes'] = (len(AltTestNamesList)*len(AltUnitNamesList))
    return data

(68, 6)


In [ ]:
test_dict = dict(zip(df['TestID'], df['TestName']))

dummy_sent_name = {
    '1': "sentance not found",
    '2': "$TNTNTN$	120/82 MMHG	$UTUTUT$",
    '3':"$TNTNTN$	5.21	$UTUTUT$	4.5-5.5	Electrical Impedence",
    '4':"$TNTNTN$	13.8	13.0 - 17.0	$UTUTUT$",
    '5':"$TNTNTN$	43.6	40 - 50	$UTUTUT$",
    '6':"$TNTNTN$	92.0	83 - 101	$UTUTUT$",
    '7':"$TNTNTN$	30.5	$UTUTUT$	31.5-34.5",
    '8':"$TNTNTN$	13.7	11.6 - 14.0	$UTUTUT$",
    '9':"$TNTNTN$	40.70	$UTUTUT$	39.0-46.0",
    '10':"$TNTNTN$	6.38	4.0 - 10.0	$UTUTUT$",
    '11':"$TNTNTN$	59	40 - 80	$UTUTUT$",
    '12':"$TNTNTN$	33	20 - 40	$UTUTUT$",
    '13':"$TNTNTN$	7	2.0 - 10.0	$UTUTUT$",
    '14':"$TNTNTN$	1	1.0 - 6.0	$UTUTUT$",
    '15':"$TNTNTN$	0	0 - 2	$UTUTUT$",
    '16':"$TNTNTN$	223000	$UTUTUT$	150000-410000	Electrical impedence",
    '17':"$TNTNTN$	3	$UTUTUT$	0-15	Modified Westergren",
    '18':"$TNTNTN$	98.0	$UTUTUT$	70-110",
    '19':"$TNTNTN$ 2	103	$UTUTUT$	70-140	HEXOKINASE",
    '20':"$TNTNTN$ 2	103	$UTUTUT$	70-140",
    '21':"$TNTNTN$	5.6	Non-diabetic: < 5.7	$UTUTUT$",
    '22':"$TNTNTN$	141	< 200 Desirable	$UTUTUT$",
    '23':"$TNTNTN$	36	Low	Low HDL Cholesterol <40	$UTUTUT$",
    '24':"$TNTNTN$	93.00	Optimal : < 100	$UTUTUT$",
    '25':"$TNTNTN$	245	$UTUTUT$	<150	Enzymatic",
    '26':"sentance not found",
    '27':"sentance not found",
    '28':"$TNTNTN$	123	Optimal : <130 $UTUTUT$",
    '29':"$TNTNTN$	1.16	0.80-2.00 $UTUTUT$",
    '30':"$TNTNTN$	8.20	4.6-12.0 $UTUTUT$",
    '31':"$TNTNTN$	1.74	0.27-4.20 $UTUTUT$",
    '32':"$TNTNTN$	17.34	7 - 18	$UTUTUT$",
    '33':"$TNTNTN$	0.67	0.67 - 1.17	$UTUTUT$",
    '34':"$TNTNTN$	5.60	3.5 - 7.2	$UTUTUT$",
    '35':"$TNTNTN$	37.10	17- 43	$UTUTUT$",
    '36':"$TNTNTN$	9.70	8.8 - 10.6	$UTUTUT$",
    '37':"$TNTNTN$	2.39	$UTUTUT$	2.6-4.5",
    '38':"$TNTNTN$	33.68	Deficiency: < 20 $UTUTUT$",
    '39':"$TNTNTN$	143	$UTUTUT$	107.2-653.3	CLIA",
    '40':"$TNTNTN$	291	158-538 $UTUTUT$",
    '41':"$TNTNTN$	88.6	33-193 $UTUTUT$",
    '42':"$TNTNTN$	6.88	$UTUTUT$	15 - 45",
    '43':"$TNTNTN$	140	136-149 $UTUTUT$",
    '44':"$TNTNTN$	4.4	3.5-5.3 $UTUTUT$",
    '45':"$TNTNTN$	102.2	$UTUTUT$	98-107",
    '46':"$TNTNTN$	19.0	$UTUTUT$	<35",
    '47':"$TNTNTN$	27.0	$UTUTUT$	<45",
    '48':"$TNTNTN$ 24.0	$UTUTUT$	7-50",
    '49':"$TNTNTN$	0.10	$UTUTUT$	0.1-0.5",
    '50':"$TNTNTN$	0.30	$UTUTUT$	0.1-1.0",
    '51':"$TNTNTN$	0.40	$UTUTUT$	0.2-1.1",
    '52':"$TNTNTN$	76	$UTUTUT$	32-91",
    '53':"$TNTNTN$	4.5	$UTUTUT$	3.5-5.0",
    '54':"$TNTNTN$	2.5	$UTUTUT$	2.3-3.5",
    '55':"sentance not found",
    '56':"$TNTNTN$	7.00	$UTUTUT$	6.5-8.1",
    '61':"$TNTNTN$	7.00	$UTUTUT$	6.5-8.1",
    '63':"$TNTNTN$	0.89	$UTUTUT$	<2.5",
    '64':"$TNTNTN$	31.7	$UTUTUT$	27-32",
    '65':"$TNTNTN$	42.6	$UTUTUT$	10 - 40",
    '67':"$TNTNTN$	42.6	$UTUTUT$	10 - 40",
    '68':"$TNTNTN$	42.6	$UTUTUT$	10 - 40",
    '69':"$TNTNTN$	184.18	22-322 $UTUTUT$",
    '77':"$TNTNTN$ 1-2 $UTUTUT$ 0-5 Microscopy",
    '78':"$TNTNTN$ 2-4 $UTUTUT$ <10 MICROSCOPY",
    '79':"$TNTNTN$ NIL $UTUTUT$ 0-2 MICROSCOPY",
    '80':"$TNTNTN$	8.20	4.6-12.0 $UTUTUT$",
    '81':"$TNTNTN$	8.20	4.6-12.0 $UTUTUT$"
}

In [ ]:
def get_all_sent_list(TotalOutcomes, AltTestNamesList, AltUnitNamesList, sent):
    all_sent_for_test = []
    for test_nm in AltTestNamesList:
        for unit in AltUnitNamesList:
            # sent = f"{test_nm}	245	{unit}	<150	Enzymatic"
            # sent = dummy_sent_name[str(test_id)]
            sent1 = sent.replace('$TNTNTN$', test_nm).replace('$UTUTUT$', unit)
            #print(sent)
            len_test = len(test_nm)
            mid_sent = sent1[len_test:sent1.index(unit)]
            # print(mid_sent)
            unit_start = len_test+len(mid_sent)
            unit_end = unit_start+len(unit)
            sent_ = [sent1, {'entities': [[0, len_test, 'TEST NAME'], [unit_start, unit_end, 'UNIT']]}]
            all_sent_for_test.append(sent_)

    if TotalOutcomes==len(all_sent_for_test):
        print(len(all_sent_for_test))
        print("All test sentence created......")
    else:
        print("Some test sentance missing")
        print("TotalOutcomes: ", TotalOutcomes)
        print("length of list 'all_sent_for_test': ", len(all_sent_for_test))

    return all_sent_for_test

def update_input_file(all_sent_for_test):
    # Define the filename
    filename = 'training_data.json'

    # Initialize TRAIN_DATA as an empty list
    TRAIN_DATA = []
    # Open the file using 'with' statement to ensure it is properly closed
    with open(filename, 'r') as f:
        # Load the JSON data into TRAIN_DATA
        TRAIN_DATA = json.load(f)

    # Before append length of ann list
    bf = len(TRAIN_DATA['annotations'])
    print("Before update length of annotations: ", bf)
    # append to previous annotations

    TRAIN_DATA['annotations'].extend(all_sent_for_test)
    # After append length of ann list
    af = len(TRAIN_DATA['annotations'])
    print("After update lenthg of annotations: ", af)
    print("how many record added: ",(af- bf))

    # Save the updated data back to the same dictionary
    with open(filename, 'w') as f:
        json.dump(TRAIN_DATA, f)
    print("training_data updated..")


def check_tn_un(all_sent_for_test, idx_):
    a1 = all_sent_for_test[idx_]
    print(a1)
    tnm = a1[1]['entities'][0]
    unm = a1[1]['entities'][1]
    sent = a1[0]
    tn = sent[tnm[0]:tnm[1]]
    un = sent[unm[0]:unm[1]]
    print(f"TestName: '{tn}', UnitName:'{un}'")

In [ ]:
# update test_dict like test_dict[2]=['BP', '$TNTNTN$\t120/82 MMHG\t$UTUTUT$']
for i, j in dummy_sent_name.items():
    if test_dict[int(i)]:
        # print(test_dict[int(i)])
        test_dict[int(i)] = [test_dict[int(i)], j]
    else:
        print("888")

In [ ]:
test_dict

{1: ['BMI', 'sentance not found'],
 2: ['BP', '$TNTNTN$\t120/82 MMHG\t$UTUTUT$'],
 3: ['RED BLOOD CELL',
  '$TNTNTN$\t5.21\t$UTUTUT$\t4.5-5.5\tElectrical Impedence'],
 4: ['HEMOGLOBIN', '$TNTNTN$\t13.8\t13.0 - 17.0\t$UTUTUT$'],
 5: ['HEMATOCRIT', '$TNTNTN$\t43.6\t40 - 50\t$UTUTUT$'],
 6: ['Mean Corpuscular Volume', '$TNTNTN$\t92.0\t83 - 101\t$UTUTUT$'],
 7: ['Mean Corpuscular Haemoglobin Concentration',
  '$TNTNTN$\t30.5\t$UTUTUT$\t31.5-34.5'],
 8: ['RDW-CV', '$TNTNTN$\t13.7\t11.6 - 14.0\t$UTUTUT$'],
 9: ['RDW-SD', '$TNTNTN$\t40.70\t$UTUTUT$\t39.0-46.0'],
 10: ['WHITE BLOOD CELL', '$TNTNTN$\t6.38\t4.0 - 10.0\t$UTUTUT$'],
 11: ['NEUTROPHILS', '$TNTNTN$\t59\t40 - 80\t$UTUTUT$'],
 12: ['LYMPHOCYTES', '$TNTNTN$\t33\t20 - 40\t$UTUTUT$'],
 13: ['MONOCYTES', '$TNTNTN$\t7\t2.0 - 10.0\t$UTUTUT$'],
 14: ['EOSINOPHILS', '$TNTNTN$\t1\t1.0 - 6.0\t$UTUTUT$'],
 15: ['BASOPHILS', '$TNTNTN$\t0\t0 - 2\t$UTUTUT$'],
 16: ['PLATELET',
  '$TNTNTN$\t223000\t$UTUTUT$\t150000-410000\tElectrical impedence'],
 1

In [ ]:
for test_id, value_list in test_dict.items():
    if "sentance not found"  in value_list:
        print("sentance not found", test_id)
    else:
        print("sentance found", test_id)
        data = get_data(test_id )
        AltTestNamesList = data['AltTestNamesList']
        AltUnitNamesList = data['AltUnitNamesList']
        TotalOutcomes = data['TotalOutcomes']
        print("TotalOutcomes:", TotalOutcomes)
        sent = value_list[1]
        print("TestSent: ", sent)
        all_sent_for_test = get_all_sent_list(TotalOutcomes, AltTestNamesList, AltUnitNamesList, sent)
        update_input_file(all_sent_for_test)
        print("__"*10)
        check_tn_un(all_sent_for_test, np.random.randint(len(all_sent_for_test)))
        check_tn_un(all_sent_for_test, np.random.randint(len(all_sent_for_test)))
        print("__"*10)
    print("_______"*10)


sentance not found 1
______________________________________________________________________
sentance found 2
TotalOutcomes: 1
TestSent:  $TNTNTN$	120/82 MMHG	$UTUTUT$
1
All test sentence created......
Before update length of annotations:  20
After update lenthg of annotations:  21
how many record added:  1
training_data updated..
____________________
['BP\t120/82 MMHG\tmmHg', {'entities': [[0, 2, 'TEST NAME'], [15, 19, 'UNIT']]}]
TestName: 'BP', UnitName:'mmHg'
['BP\t120/82 MMHG\tmmHg', {'entities': [[0, 2, 'TEST NAME'], [15, 19, 'UNIT']]}]
TestName: 'BP', UnitName:'mmHg'
____________________
______________________________________________________________________
sentance found 3
TotalOutcomes: 378
TestSent:  $TNTNTN$	5.21	$UTUTUT$	4.5-5.5	Electrical Impedence
378
All test sentence created......
Before update length of annotations:  21
After update lenthg of annotations:  399
how many record added:  378
training_data updated..
____________________
['R.B.C. Count\t5.21\t/cu.mm\t4.5-5.5\t

In [ ]:

def load_input_data():
    # Define the filename
    filename = 'training_data.json'

    # Initialize TRAIN_DATA as an empty list
    TRAIN_DATA = []
    # Open the file using 'with' statement to ensure it is properly closed
    with open(filename, 'r') as f:
        # Load the JSON data into TRAIN_DATA
        TRAIN_DATA = json.load(f)
    return TRAIN_DATA

In [ ]:
data = load_input_data()

In [ ]:
len(data['annotations'])

3490

In [ ]:
all_sent_for_test = data['annotations']
for i in range(3400, len(all_sent_for_test)):
    print(i)
    check_tn_un(all_sent_for_test, i)
    print("_______"*10)

3400
['VLDL\t42.6\tmg/d L\t10 - 40', {'entities': [[0, 4, 'TEST NAME'], [10, 16, 'UNIT']]}]
TestName: 'VLDL', UnitName:'mg/d L'
______________________________________________________________________
3401
['VLDL\t42.6\tmg / dl\t10 - 40', {'entities': [[0, 4, 'TEST NAME'], [10, 17, 'UNIT']]}]
TestName: 'VLDL', UnitName:'mg / dl'
______________________________________________________________________
3402
['VLDL\t42.6\tmg/L\t10 - 40', {'entities': [[0, 4, 'TEST NAME'], [10, 14, 'UNIT']]}]
TestName: 'VLDL', UnitName:'mg/L'
______________________________________________________________________
3403
['VERY LOW DENSINTY\t42.6\tmg/dL\t10 - 40', {'entities': [[0, 17, 'TEST NAME'], [23, 28, 'UNIT']]}]
TestName: 'VERY LOW DENSINTY', UnitName:'mg/dL'
______________________________________________________________________
3404
['VERY LOW DENSINTY\t42.6\tmg%\t10 - 40', {'entities': [[0, 17, 'TEST NAME'], [23, 26, 'UNIT']]}]
TestName: 'VERY LOW DENSINTY', UnitName:'mg%'
________________________________

In [ ]:
test_dict

{1: ['BMI', 'sentance not found'],
 2: ['BP', '$TNTNTN$\t120/82 MMHG\t$UTUTUT$'],
 3: ['RED BLOOD CELL',
  '$TNTNTN$\t5.21\t$UTUTUT$\t4.5-5.5\tElectrical Impedence'],
 4: ['HEMOGLOBIN', '$TNTNTN$\t13.8\t13.0 - 17.0\t$UTUTUT$'],
 5: ['HEMATOCRIT', '$TNTNTN$\t43.6\t40 - 50\t$UTUTUT$'],
 6: ['Mean Corpuscular Volume', '$TNTNTN$\t92.0\t83 - 101\t$UTUTUT$'],
 7: ['Mean Corpuscular Haemoglobin Concentration',
  '$TNTNTN$\t30.5\t$UTUTUT$\t31.5-34.5'],
 8: ['RDW-CV', '$TNTNTN$\t13.7\t11.6 - 14.0\t$UTUTUT$'],
 9: ['RDW-SD', '$TNTNTN$\t40.70\t$UTUTUT$\t39.0-46.0'],
 10: ['WHITE BLOOD CELL', '$TNTNTN$\t6.38\t4.0 - 10.0\t$UTUTUT$'],
 11: ['NEUTROPHILS', '$TNTNTN$\t59\t40 - 80\t$UTUTUT$'],
 12: ['LYMPHOCYTES', '$TNTNTN$\t33\t20 - 40\t$UTUTUT$'],
 13: ['MONOCYTES', '$TNTNTN$\t7\t2.0 - 10.0\t$UTUTUT$'],
 14: ['EOSINOPHILS', '$TNTNTN$\t1\t1.0 - 6.0\t$UTUTUT$'],
 15: ['BASOPHILS', '$TNTNTN$\t0\t0 - 2\t$UTUTUT$'],
 16: ['PLATELET',
  '$TNTNTN$\t223000\t$UTUTUT$\t150000-410000\tElectrical impedence'],
 1

In [ ]:

test_id = 3
data = get_data(test_id )

AltTestNamesList = data['AltTestNamesList']
AltUnitNamesList = data['AltUnitNamesList']
TotalOutcomes = data['TotalOutcomes']
print(TotalOutcomes)

all_sent_for_test = []
for test_nm in AltTestNamesList:
    for unit in AltUnitNamesList:
        # sent = f"{test_nm}	245	{unit}	<150	Enzymatic"
        sent = dummy_sent_name[str(test_id)]
        sent = sent.replace('$TNTNTN$', test_nm).replace('$UTUTUT$', unit)
        len_test = len(test_nm)
        mid_sent = sent[len_test:sent.index(unit)]
        unit_start = len_test+len(mid_sent)
        unit_end = unit_start+len(unit)
        sent_ = [sent, {'entities': [[0, len_test, 'TEST NAME'], [unit_start, unit_end, 'UNIT']]}]
        all_sent_for_test.append(sent_)

if TotalOutcomes==len(all_sent_for_test):
    print(len(all_sent_for_test))
    print("All test sentence created......")
else:
    print("Some test sentance missing")
    print("TotalOutcomes: ", TotalOutcomes)
    print("length of list 'all_sent_for_test': ", len(all_sent_for_test))

378
378
All test sentence created......


In [ ]:


def check_tn_un(all_sent_for_test, idx_):
    a1 = all_sent_for_test[idx_]
    print(a1)
    tnm = a1[1]['entities'][0]
    unm = a1[1]['entities'][1]
    sent = a1[0]
    tn = sent[tnm[0]:tnm[1]]
    un = sent[unm[0]:unm[1]]
    print(f"TestName: '{tn}', UnitName:'{un}'")


check_id = 201
check_tn_un(all_sent_for_test,check_id)

['Red Blood Cell (RBC)\t5.21\t10^6/µl\t4.5-5.5\tElectrical Impedence', {'entities': [[0, 20, 'TEST NAME'], [26, 33, 'UNIT']]}]
TestName: 'Red Blood Cell (RBC)', UnitName:'10^6/µl'


In [ ]:
# Define the filename
filename = 'training_data.json'

# Initialize TRAIN_DATA as an empty list
TRAIN_DATA = []
# Open the file using 'with' statement to ensure it is properly closed
with open(filename, 'r') as f:
    # Load the JSON data into TRAIN_DATA
    TRAIN_DATA = json.load(f)

In [ ]:
TRAIN_DATA

{'classes': ['TEST NAME', 'UNIT'],
 'annotations': [['RBC COUNT\t5.12\tMillion/cu.mm\t3.8-4.8\tElectrical Impedence\r',
   {'entities': [[0, 9, 'TEST NAME'], [15, 28, 'UNIT']]}],
  ['HAEMOGLOBIN\t13.9\tg/dL\t12-15\tSpectrophotometer\r',
   {'entities': [[0, 11, 'TEST NAME'], [17, 21, 'UNIT']]}],
  ['TOTAL IRON BINDING CAPACITY ( TIBC)\t325\tµg/dL\t261-462\tTPTZ AND NITROSO-\r',
   {'entities': [[0, 35, 'TEST NAME'], [40, 45, 'UNIT']]}],
  ['IRON\t89.0\tµg/dL\t60-180\tTPTZ\r',
   {'entities': [[0, 4, 'TEST NAME'], [10, 15, 'UNIT']]}],
  ['GAMMA GLUTAMYL TRANSPEPTIDASE\t26.00\tU/L\t<38\tIFCC\r',
   {'entities': [[0, 29, 'TEST NAME'], [36, 39, 'UNIT']]}],
  ['BILIRUBIN CONJUGATED (DIRECT)\t0.13\tmg/dL\t<0.2\tDPD\r',
   {'entities': [[0, 29, 'TEST NAME'], [35, 40, 'UNIT']]}],
  ['PCV\t40.50\t%\t36-46\tElectronic pulse &\r',
   {'entities': [[0, 3, 'TEST NAME'], [10, 11, 'UNIT']]}],
  ['MCV\t79.1\tfL\t83-101\tCalculated\r',
   {'entities': [[0, 3, 'TEST NAME'], [9, 11, 'UNIT']]}],
  ['MCHC\

In [ ]:
# Define the filename
filename = 'training_data.json'

# Initialize TRAIN_DATA as an empty list
TRAIN_DATA = []
# Open the file using 'with' statement to ensure it is properly closed
with open(filename, 'r') as f:
    # Load the JSON data into TRAIN_DATA
    TRAIN_DATA = json.load(f)

# Before append length of ann list
bf = len(TRAIN_DATA['annotations'])
print(len(TRAIN_DATA['annotations']))
# append to previous annotations

TRAIN_DATA['annotations'].extend(all_sent_for_test)
# After append length of ann list
af = len(TRAIN_DATA['annotations'])
print(len(TRAIN_DATA['annotations']))
print((af- bf))

# Save the updated data back to the same dictionary
with open(filename, 'w') as f:
    json.dump(TRAIN_DATA, f)

In [ ]:
def update_input_file(all_sent_for_test):
    # Define the filename
    filename = 'training_data.json'

    # Initialize TRAIN_DATA as an empty list
    TRAIN_DATA = []
    # Open the file using 'with' statement to ensure it is properly closed
    with open(filename, 'r') as f:
        # Load the JSON data into TRAIN_DATA
        TRAIN_DATA = json.load(f)

    # Before append length of ann list
    bf = len(TRAIN_DATA['annotations'])
    print("Before update length of annotations: ", bf)
    # append to previous annotations

    TRAIN_DATA['annotations'].extend(all_sent_for_test)
    # After append length of ann list
    af = len(TRAIN_DATA['annotations'])
    print("After update lenthg of annotations: ", af)
    print("how many record added: ",(af- bf))

    # Save the updated data back to the same dictionary
    with open(filename, 'w') as f:
        json.dump(TRAIN_DATA, f)

In [ ]:
!pip install -U spacy -q
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.7.4                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.58+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.7.1)        

